In [1]:
Names: ['namestudent1', 'namestudent2'] 
Studentnumbers: ['studentnumberstudent1', 'studentnumberstudent2']

In [2]:
# Download the data
# Dataset with 10K instances
! gdown "https://drive.google.com/uc?id=1cj-CzkY6QZUe42ky64GI5CSSg7-K40N5"
# Dataset with 100k instances
! gdown "https://drive.google.com/uc?id=1MbWGXLawE3VTxP1XgNpj8uEo1VHPq12B"
! pip install sklearn
! pip install pandas

Downloading...
From: https://drive.google.com/uc?id=1cj-CzkY6QZUe42ky64GI5CSSg7-K40N5
To: /Users/daanschoppink/Downloads/loan_default_10K.csv
100%|██████████████████████████████████████| 48.3M/48.3M [00:02<00:00, 17.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1MbWGXLawE3VTxP1XgNpj8uEo1VHPq12B
To: /Users/daanschoppink/Downloads/loan_default_100K.csv
100%|████████████████████████████████████████| 484M/484M [00:29<00:00, 16.2MB/s]
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installa

### Part 1: Import the data

# Applied Machine learning
## Practical Assignment 2

### Important Notes:
1. Submit through **Canvas** before 11:59pm on Tuesday, May 17, 2022
2. No late homework will be accepted
3. This is a group-of-two assignment
4. The submitted file should be in ipynb format
5. The assignment is worth it 10 points
6. For questions, please use the discussion part of Canvas (English only!)
7. The indication **optional** means that the question is optional; you won't lose any points if you do not do that part of the assignment, nor will you gain if you do it.

### Software:
We will be using Python programming language throughout this course. Further we will be using:
+ IPython Notebooks (as an environment)
+ Numpy
+ Pandas
+ Scikit-learn

### Background:

This practical assignment will be covering logistic regression, neural networks, support vector machines and evaluation of classifiers. 

For the assignment, please download a dataset on Load Defaults. You are provided with two datasets:
1. [Dataset](https://drive.google.com/open?id=1cj-CzkY6QZUe42ky64GI5CSSg7-K40N5) with 10,000 instances 
2. [Dataset](https://drive.google.com/open?id=1MbWGXLawE3VTxP1XgNpj8uEo1VHPq12B) with 100,000 instances

In principle you should work on the second, larger dataset, but if you face scaling computational issues then better work with the first, smaller dataset.

This data corresponds to a set of financial transactions associated with individuals. The data has been standardized, de-trended, and anonymized. You are provided with thousands of observations and nearly 800 features. Each observation (instance) is independent from the previous. 

For each observation, it was recorded whether a default was triggered (i.e. whether an individual could not pay back the loan). In case of a default, the loss for the bank was measured. This quantity lies between 0 and 100. If the loan did not default, the loss for the bank was 0. You are asked to predict whether a loan will default or not for each individual in the test set.

Missing feature values have been kept as is, so that the competing teams can really use the maximum data available, implementing a strategy to fill the gaps if desired. Consider all variables continuous, even though some variables may be categorical (e.g. f776 and f777).

The goal of the machine learning algorithm will be to predict whether a loan will default, given a set of features. For privacy reasons the feature names are not provided.

**Important Note**: This second assignment is not as instructive as the first assignment. The first assignment guided you step-by-step through all the preprocessing, training-validation-testing setup, etc. This assignment does not do so, but it leaves it up to you to decide how to use the data and design your experiments.

In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv('loan_default_10K.csv', sep=",", header=0, dtype=np.float64)

# Drop the observations that contain missing values
dfn = df.dropna(0, how='any')

# Consider only a handful of features to start with; you can extend to the full set later on.
X = dfn.loc[:,'f400':'f500'].values

# Generate the labels; if 'loss' is zero the this indicates the negative class, class 0, i.e. no default;
# if 'loss' is possitive this indicates the positive class, class 1, i.e. there is a loan default;
y = [ bool(y) for y in dfn.loc[:,'loss'].values ]

# Split the data into train, validation, and test.
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.20)

<ipython-input-17-2fe94ea43ea5>:8: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only
  dfn = df.dropna(0, how='any')


In [74]:
test_pred = [True, True, False, True, False, False, True, True, True, True, True, True]
test_true = [True, False, False, True, True, False, False, True, False, False, False, False]

print(y_test.count(True))
print(y_test.count(False))

93
781


### Part 2: Evaluation measures (2pts)
In what follows you should implement a number of evaluation measures. You need to implement these from scratch, meaning that it is not allowed to call any scikit-learn function, or any other API function that implements the method for you.

* Implement a function that produces the contigency matrix, i.e. True Positives, False Positives, True Negatives, False Negative

In [18]:
def contigency_matrix(true_y, predicted_y):
    # YOUR CODE HERE, Create TP, FP, TN, FN
    TP, FP, TN, FN = 0, 0, 0, 0
    for i, j in zip(true_y, predicted_y):
        TP += 1 if (i == True and i == j) else 0
        FP += 1 if (i == False and j == True) else 0
        TN += 1 if (i == False and i == j) else 0
        FN += 1 if (i == True and j == False) else 0
    
    return np.array(([TP, FP], [TN, FN]))

* Implement a function that computes accuracy (without using any built-in accuracy function)

In [19]:
def accuracy(true_y, predicted_y):
    M = contigency_matrix(true_y, predicted_y)
    TP, FP, TN, FN = M[0,0], M[0,1], M[1,0], M[1,1]
    
    return (TP + TN) / (TP + TN + FP + FN)

* Implement a function that computes precision for one class (without using any built-in precision function)

In [20]:
def precision(true_y, predicted_y):
    M = contigency_matrix(true_y, predicted_y)
    TP, FP = M[0,0], M[0,1]
    
    return TP / (TP + FP)

* Implement a function that computes recall for one class (without using any built-in recall function)

In [21]:
def recall(true_y, predicted_y):
    M = contigency_matrix(true_y, predicted_y)
    TP, FN = M[0,0], M[1,1]
    
    return TP / (TP + FN)
   

* Implement a function that computes f1 for one class(without using any built-in f1 function)

In [22]:
def f1(true_y, predicted_y):
    p = precision(true_y, predicted_y)
    r = recall(true_y, predicted_y)
    
    return 2 * ((p * r) / (p + r))

### Part 3: Algorithms
Compare the performance of Logistic Regression, SVMs and Neural Networks

##### Logistic Regression (Lecture 3) (2pts)

+ Train and test a logistic regression model
    + Construct a table with each row being a different value of the regularization parameter and each column the aforementioned measures
    + Explain your findings and select the optimal model
    + Report the performance of the optimal model

In [30]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

train_scaler = StandardScaler().fit(X_train)
X_train_scaled = train_scaler.transform(X_train)

test_scaler = StandardScaler().fit(X_test)
X_test_scaled = test_scaler.transform(X_test)

# X_train = StandardScaler().fit_transform(X_train)
# X_test = StandardScaler().fit_transform(X_test)

In [31]:
from sklearn.linear_model import LogisticRegression

def create_LG(X, y, c: float = 1.0, balance = None):
    return LogisticRegression(max_iter = 5000, C = c,  class_weight= balance).fit(X, y)

In [41]:
c_list = np.arange(0.1, 5.0, 0.1)
M = []
M_balanced = []
for c in c_list:
    model = [create_LG(X_train_scaled, y_train, c, None), 
             create_LG(X_train_scaled, y_train, c, 'balanced')]
    
    y_pred = [model[0].predict(X_test_scaled),model[1].predict(X_test_scaled)] 
    
    acc = [accuracy(y_test, y_pred[0]), accuracy(y_test, y_pred[1])]
    pre = [precision(y_test, y_pred[0]), precision(y_test, y_pred[1])]
    rec = [recall(y_test, y_pred[0]), recall(y_test, y_pred[1])]
    f1s = [f1(y_test, y_pred[0]), f1(y_test, y_pred[1])]
    
    M.append([c, acc[0], pre[0], rec[0] ,f1s[0]])
    M_balanced.append([c, acc[1], pre[1], rec[1] ,f1s[1]])

<ipython-input-20-18a030370a7f>:5: RuntimeWarning: invalid value encountered in long_scalars
  return TP / (TP + FP)
<ipython-input-20-18a030370a7f>:5: RuntimeWarning: invalid value encountered in long_scalars
  return TP / (TP + FP)
<ipython-input-20-18a030370a7f>:5: RuntimeWarning: invalid value encountered in long_scalars
  return TP / (TP + FP)
<ipython-input-20-18a030370a7f>:5: RuntimeWarning: invalid value encountered in long_scalars
  return TP / (TP + FP)
<ipython-input-20-18a030370a7f>:5: RuntimeWarning: invalid value encountered in long_scalars
  return TP / (TP + FP)
<ipython-input-20-18a030370a7f>:5: RuntimeWarning: invalid value encountered in long_scalars
  return TP / (TP + FP)
<ipython-input-20-18a030370a7f>:5: RuntimeWarning: invalid value encountered in long_scalars
  return TP / (TP + FP)
<ipython-input-20-18a030370a7f>:5: RuntimeWarning: invalid value encountered in long_scalars
  return TP / (TP + FP)
<ipython-input-20-18a030370a7f>:5: RuntimeWarning: invalid value

In [47]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

LG_table = pd.DataFrame(M, columns = ['Reg param','Accuracy','Precision', 'Recall', 'f1'])
LG_table_balanced = pd.DataFrame(M_balanced, columns = ['Reg param','Accuracy','Precision', 'Recall', 'f1'])

print("Imbalanced data: ")
display(LG_table)
print("\n")
print("Balanced data: ")
display(LG_table_balanced)

Imbalanced data: 


,Reg param,Accuracy,Precision,Recall,f1
0,0.1,0.893593,NaN,0.0,NaN
1,0.2,0.893593,NaN,0.0,NaN
2,0.3,0.893593,NaN,0.0,NaN
3,0.4,0.893593,NaN,0.0,NaN
4,0.5,0.893593,NaN,0.0,NaN
5,0.6,0.893593,NaN,0.0,NaN
6,0.7,0.893593,NaN,0.0,NaN
7,0.8,0.893593,NaN,0.0,NaN
8,0.9,0.893593,NaN,0.0,NaN
9,1.0,0.893593,NaN,0.0,NaN




Balanced data: 


,Reg param,Accuracy,Precision,Recall,f1
0,0.1,0.582380,0.145833,0.602151,0.234801
1,0.2,0.581236,0.145455,0.602151,0.234310
2,0.3,0.576659,0.143959,0.602151,0.232365
3,0.4,0.572082,0.142494,0.602151,0.230453
4,0.5,0.567506,0.141058,0.602151,0.228571
5,0.6,0.561785,0.141089,0.612903,0.229376
6,0.7,0.557208,0.137931,0.602151,0.224449
7,0.8,0.551487,0.136253,0.602151,0.222222
8,0.9,0.546911,0.131387,0.580645,0.214286
9,1.0,0.544622,0.130751,0.580645,0.213439


In [48]:
acc_col = LG_table_balanced["Accuracy"]
pre_col = LG_table_balanced["Precision"]
re_col = LG_table_balanced["Recall"]
f1_col = LG_table_balanced['f1']
print(acc_col.idxmax())
print(pre_col.idxmax())
print(re_col.idxmax())
print(f1_col.idxmax())

0
0
5
0


* Explain what you observe regarding the positive class; i.e. the performance of the algorithm in predicting defaults. Explain why is this happening.

Input is rarely classefied as the positive class, this is due to the data set being imbalanced in favor of the negative class. If no balance method is used, calculating the aforementioned measures result in NaN outputs. LogisticRegression() from Sklearn offers a paramter, class_weight,  to balance the weights, partially migating the imbalance in the data set.

Looking at both tables, unbalanced and balanced, the optimal model is a model with a low regularization parameter. Testing with different regularization parameter intervals it also showed that a low regularization parameter resulted in better performace. 

There are a number of ways to fix the problem you have observed above. Here we will consider two of them: downsampling and upsampling. In an ideal situation you will like your dataset to be balanced, i.e. to have the same number of instances for the positive and the negative class.

**Downsampling**: Let's assume that the positive class has *n1* instances, while the negative class *n2* instances, where *n2* is much bigger than *n1*. One solution is to create a new training set for which from the *n2* instances of the negative class you sample *n1* of them only to include in your training set; hence now you have *n1* + *n1* training instances.

**Upsampling**: Let's assume that the positive class has *n1* instances, while the negative class *n2* instances, where *n2* is much bigger than *n1*. Another solution is to create a new training set for which you create  *n2* instances of the positive class. To do so you sample *n2* instances from the *n1* instance, with replacement. With replacement means that you allow the same instance to be sampled multiple times; hence now you have *n2* + *n2* training instances.

#### Downsampling (OPTIONAL – If you wish to skip downsampling continue to Neural Networks further below)


* Implement a function for downsampling (**optional**)

In [15]:
def downsample(y_train):
    # y_train is the 1d matrix of the labels in your training data, e.g.
    #       0     1     2     3     4   5     6     7     8   ... 
    # y = [True False False False True True False False False ... False]
    #
    # the function returns the position of the training data to be considered for the final training set.
    # e.g. if you decide from the True instances to select 0, 4 and 5, while from the False instances 1, 3, and 8
    # the outcome of the function will be [0, 1, 3, 4, 5, 8] (= sampled_indexes)
    
    # your code goes here
    
    return sampled_indexes
    
def new_training_set(X_train, y_train, sampled_indexes):
    # your code goes here

SyntaxError: unexpected EOF while parsing (<ipython-input-15-2ac96c953aaa>, line 15)

* Test the performance of logistic regression using the new training set, and report your conclusions (**optional**)

In [ ]:
# YOUR CODE HERE

# The last few questions below are not optional!
If you did not finish the optional downsampling, just go through with the data created before

##### SVMs (Lecture 4) (2pts)

+ Train and test a Support Vector Machine model
    + Construct a table with each row being a different configuration of the SVM algorithm (play with the regularization parameter, and the kernel function – use linear, poly, and rbf) and each column the evaluation measures
    + Explain your findings and select the optimal model
    + Report the performance of the optimal model

In [70]:
from sklearn import svm
def create_SVM(X, y, k, c: float = 1.0, balance = None):
    return svm.SVC(kernel = k, C = c,  class_weight= balance).fit(X, y)

In [71]:
c_list = np.arange(0.1, 5.0, 0.1)
SVM_M = []
SVM_M_balanced = []
for c in c_list:
    model = [create_SVM(X_train_scaled, y_train, 'linear', c, None), 
             create_SVM(X_train_scaled, y_train, 'linear', c, 'balanced')]
    
    y_pred = [model[0].predict(X_test_scaled),model[1].predict(X_test_scaled)] 
    
    acc = [accuracy(y_test, y_pred[0]), accuracy(y_test, y_pred[1])]
    pre = [precision(y_test, y_pred[0]), precision(y_test, y_pred[1])]
    rec = [recall(y_test, y_pred[0]), recall(y_test, y_pred[1])]
    f1s = [f1(y_test, y_pred[0]), f1(y_test, y_pred[1])]
    
    SVM_M.append([c, acc[0], pre[0], rec[0] ,f1s[0]])
    SVM_M_balanced.append([c, acc[1], pre[1], rec[1] ,f1s[1]])

<ipython-input-20-18a030370a7f>:5: RuntimeWarning: invalid value encountered in long_scalars
  return TP / (TP + FP)
<ipython-input-20-18a030370a7f>:5: RuntimeWarning: invalid value encountered in long_scalars
  return TP / (TP + FP)
<ipython-input-20-18a030370a7f>:5: RuntimeWarning: invalid value encountered in long_scalars
  return TP / (TP + FP)
<ipython-input-20-18a030370a7f>:5: RuntimeWarning: invalid value encountered in long_scalars
  return TP / (TP + FP)
<ipython-input-20-18a030370a7f>:5: RuntimeWarning: invalid value encountered in long_scalars
  return TP / (TP + FP)
<ipython-input-20-18a030370a7f>:5: RuntimeWarning: invalid value encountered in long_scalars
  return TP / (TP + FP)
<ipython-input-20-18a030370a7f>:5: RuntimeWarning: invalid value encountered in long_scalars
  return TP / (TP + FP)
<ipython-input-20-18a030370a7f>:5: RuntimeWarning: invalid value encountered in long_scalars
  return TP / (TP + FP)
<ipython-input-20-18a030370a7f>:5: RuntimeWarning: invalid value

KeyboardInterrupt: 

In [69]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

SVM_table = pd.DataFrame(M, columns = ['Reg param','Accuracy','Precision', 'Recall', 'f1'])
SVM_table_balanced = pd.DataFrame(M_balanced, columns = ['Reg param','Accuracy','Precision', 'Recall', 'f1'])

print("Imbalanced data: ")
display(SVM_table)
print("\n")
print("Balanced data: ")
display(SVM_table_balanced)

Imbalanced data: 


,Reg param,Accuracy,Precision,Recall,f1
0,0.1,0.893593,NaN,0.0,NaN
1,0.2,0.893593,NaN,0.0,NaN
2,0.3,0.893593,NaN,0.0,NaN
3,0.4,0.893593,NaN,0.0,NaN
4,0.5,0.893593,NaN,0.0,NaN
5,0.6,0.893593,NaN,0.0,NaN
6,0.7,0.893593,NaN,0.0,NaN
7,0.8,0.893593,NaN,0.0,NaN
8,0.9,0.893593,NaN,0.0,NaN
9,1.0,0.893593,NaN,0.0,NaN




Balanced data: 


,Reg param,Accuracy,Precision,Recall,f1
0,0.1,0.582380,0.145833,0.602151,0.234801
1,0.2,0.581236,0.145455,0.602151,0.234310
2,0.3,0.576659,0.143959,0.602151,0.232365
3,0.4,0.572082,0.142494,0.602151,0.230453
4,0.5,0.567506,0.141058,0.602151,0.228571
5,0.6,0.561785,0.141089,0.612903,0.229376
6,0.7,0.557208,0.137931,0.602151,0.224449
7,0.8,0.551487,0.136253,0.602151,0.222222
8,0.9,0.546911,0.131387,0.580645,0.214286
9,1.0,0.544622,0.130751,0.580645,0.213439


<span style="color:blue">**Replace the text in this cell with your report**</span>

##### Neural Network (Lecture 5) (2pts)

+ Train and test a Neural Network model
    + Construct a table with each row being a different configuration of the network (play with the number of hidden layers, the number of neurons in each layer, and the activation function) and each column the evaluation measures
    + Report the performance of at least three different configurations
    + Explain your findings and select the optimal model
    + Justify your choice of different paramteres and architectures
    + Report the performance of the optimal model

In [ ]:
# YOUR CODE HERE

<span style="color:blue">**Replace the text in this cell with your report**</span>

#### Compare Algorithms (2pts)
* Plot the Precision-Recall curves for the best model for each one of the above algorithms, Logistic Regression, Neural Nets, and SVM.
    * Use the precision_recall_curve from scikit-learn
* Explain your findings

In [ ]:
# YOUR CODE HERE

<span style="color:blue">**Replace the text in this cell with your report**</span>